In [1]:
%cd '/content/drive/MyDrive/mulcam_project/BERT_idiom'

[WinError 3] 지정된 경로를 찾을 수 없습니다: "'/content/drive/MyDrive/mulcam_project/BERT_idiom'"
c:\githubPage\MultiCampus_TeamProject\2021_NLP_Project\Step01_Idiom_Classfier


In [ ]:
!pip install transformers

In [2]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime
import pickle

ModuleNotFoundError: No module named 'torch'

In [ ]:
with open('./TroFi_dataset.pkl', 'rb') as f:
  corpus = pickle.load(f)

In [ ]:
corpus.head()

,Label,Text
0,1,But in the short-term it will absorb a lot of ...
1,1,Shocks from one-time changes in the terms of t...
2,1,"During the past 18 months , the Houston-based ..."
3,1,"In the years since 1853 , when 4 , 058 steambo..."
4,1,Mr. Wyss of Data Resources suggested that fact...


In [ ]:
train, test = train_test_split(corpus, test_size=0.2, random_state=42)

In [ ]:
sentences = train['Text']
sentences[:10]

485     The doctor who removed the IUD destroyed Ms. M...
2170    In Mr. Kupfer 's `` Ring , '' a nuclear war ha...
3291    Ford , Chrysler and GM are responding with str...
1097    Before examining them , Mr. Araujo says he has...
258     The U.S. armed forces are being besieged these...
2632    Carl Rowan , of course , is the famous liberal...
3344    If a deal is struck , the 767s will replace ag...
1611    During 1987 , America West added 2 , 900 emplo...
1593    A slick young marketing executive at the firm ...
2688    `` If it rains three inches , we 'd go down 30...
Name: Text, dtype: object

In [ ]:
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

["[CLS] The doctor who removed the IUD destroyed Ms. Murdock-Vaughn 's files in 1979 as part of a routine office cleaning [SEP]",
 "[CLS] In Mr. Kupfer 's `` Ring , '' a nuclear war has melted most of the world and made shopping difficult [SEP]",
 '[CLS] Ford , Chrysler and GM are responding with strategies of their own , and all essentially incorporate the same truth: A lot of headaches can be prevented by planning ahead , sticking to decisions and working as a team [SEP]',
 '[CLS] Before examining them , Mr. Araujo says he has to read the MiG flight manual , an unwieldly volume he discovered was written in Chinese [SEP]',
 '[CLS] The U.S. armed forces are being besieged these days -- not by foreign military aggressors -- but by aggressive consumer-product marketers [SEP]',
 '[CLS] Carl Rowan , of course , is the famous liberal newspaper columnist and TV commentator who may have become even more famous Tuesday for pumping a bullet from a .22-caliber handgun into the wrist of Ben Neal 

In [ ]:
# 라벨 추출
labels = train['Label'].values
labels

array([0, 0, 1, ..., 1, 1, 1])

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] The doctor who removed the IUD destroyed Ms. Murdock-Vaughn 's files in 1979 as part of a routine office cleaning [SEP]
['[CLS]', 'The', 'doctor', 'who', 'removed', 'the', 'I', '##UD', 'destroyed', 'Ms', '.', 'Mur', '##dock', '-', 'Vaughn', "'", 's', 'files', 'in', '1979', 'as', 'part', 'of', 'a', 'routine', 'office', 'clean', '##ing', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 64

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,  10117,  26937,  10479,  23898,  10105,    146,  86888,
        24089,  40792,    119, 103192,  68742,    118, 102117,    112,
          187,  33646,  10106,  10675,  10146,  10668,  10108,    169,
        94650,  14301,  55911,  10230,    102,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([  101, 11061, 10835,   117,   169, 16511, 10108, 11061, 10835, 12054,
        13604,   119,   117, 12415, 10474, 11288, 26482, 29467, 10247,   119,
          123,   110, 10114,   122,   119, 10193, 25848, 51557, 33155, 14148,
        10188, 10105, 10924,   118, 18905, 69879,   119,   126, 12473, 51557,
        33155, 14148,   117, 10345, 10464, 81846, 33155, 30676, 10115, 10464,
        21128,   102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor(0)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
tensor([  101, 12845, 69302, 34598, 44687,   119, 12415, 10271, 25577, 10114,
        16868,   109, 11824, 12473, 10114, 90424, 10435, 25880, 53933, 10135,
        50683, 10108, 10250, 11426, 1587

In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

## 전처리 - 테스트셋

In [ ]:
sentences = test['Text']
sentences[:10]

1151    The program is designed to let people using pe...
538     Weyerhaeuser Co. said it has budgeted an extra...
351     Joffrey Ballet: The bicoastal company dances i...
1108    He urged the Norwegian government to press its...
1662    Mr. Petee said one man was so afraid his boss ...
605     A state judge in Manhattan ordered White & Cas...
93      Mr. Gallas says he began looking into melanin ...
358     `` I have undergraduate degrees in chemistry a...
3359    He also brought his will to throw bats , batti...
3316    It took a year and two trips to the Soviet Uni...
Name: Text, dtype: object

In [ ]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] The program is designed to let people using personal computers on a network plan meetings , share comments , track projects and examine libraries of documents [SEP]',
 '[CLS] Weyerhaeuser Co. said it has budgeted an extra $ 1 million to replace seedlings that may die [SEP]',
 "[CLS] Joffrey Ballet: The bicoastal company dances its first Manhattan season since the death of founder Robert Joffrey , whose own `` Remembrances '' pas de deux will be revived [SEP]",
 '[CLS] He urged the Norwegian government to press its rights to inspect the Dimona reactor and examine any materials , such as plutonium , possibly made from using the heavy water [SEP]',
 '[CLS] Mr. Petee said one man was so afraid his boss was going to ask him to fly that he started his own business [SEP]',
 "[CLS] A state judge in Manhattan ordered White & Case to provide an accounting of the firm 's assets as of June 30 , the day lawyers in the firm voted to dissolve their partnership and re-form without longtime par

In [ ]:
# 라벨 추출
labels = test['Label'].values
labels

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0,

In [ ]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] The program is designed to let people using personal computers on a network plan meetings , share comments , track projects and examine libraries of documents [SEP]
['[CLS]', 'The', 'program', 'is', 'designed', 'to', 'let', 'people', 'using', 'personal', 'computers', 'on', 'a', 'network', 'plan', 'meetings', ',', 'share', 'comments', ',', 'track', 'projects', 'and', 'ex', '##amine', 'libraries', 'of', 'documents', '[SEP]']


In [ ]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 64

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([  101, 10117, 12483, 10124, 15513, 10114, 13595, 11426, 13382,
       14927, 58838, 10135,   169, 17175, 13065, 47307,   117, 23867,
       61565,   117, 14310, 22846, 10111, 11419, 45980, 77297, 10108,
       21784,   102,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0])

In [ ]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
# 데이터를 파이토치의 텐서로 변환
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)

print(test_inputs[0])
print(test_labels[0])
print(test_masks[0])

tensor([  101, 10117, 12483, 10124, 15513, 10114, 13595, 11426, 13382, 14927,
        58838, 10135,   169, 17175, 13065, 47307,   117, 23867, 61565,   117,
        14310, 22846, 10111, 11419, 45980, 77297, 10108, 21784,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
tensor(0)
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])


In [ ]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [ ]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [ ]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, f1_score
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    accuracy = np.sum(pred_flat == labels_flat) / len(labels_flat)
    precision = precision_score(pred_flat, labels_flat)
    recall = recall_score(pred_flat, labels_flat)
    f1score = f1_score(pred_flat, labels_flat)
    return accuracy, precision, recall, f1score

In [ ]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels)
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy, _, _, _ = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...

  Average training loss: 0.54
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.801
  Validation took: 0:00:01

======== Epoch 2 / 4 ========
Training...

  Average training loss: 0.33
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.841
  Validation took: 0:00:01

======== Epoch 3 / 4 ========
Training...

  Average training loss: 0.21
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.838
  Validation took: 0:00:01

======== Epoch 4 / 4 ========
Training...

  Average training loss: 0.13
  Training epcoh took: 0:00:18

Running Validation...
  Accuracy: 0.816
  Validation took: 0:00:01

Training complete!


In [ ]:
#시작 시간 설정
t0 = time.time()

# 평가모드로 변경
model.eval()

# 변수 초기화
eval_loss, eval_accuracy, eval_precision, eval_recall, eval_f1score = 0, 0, 0, 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(test_dataloader):
    # 경과 정보 표시
    if step % 100 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(test_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    # 배치에서 데이터 추출
    b_input_ids, b_input_mask, b_labels = batch
    
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)
    
    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    # 출력 로짓과 라벨을 비교하여 정확도 계산
    tmp_eval_accuracy, tmp_eval_precision, tmp_eval_recall, tmp_eval_f1score = flat_accuracy(logits, label_ids)
    eval_accuracy += tmp_eval_accuracy
    eval_precision += tmp_eval_precision
    eval_recall += tmp_eval_recall
    eval_f1score += tmp_eval_f1score
    nb_eval_steps += 1

print("")
print("Precision: {0:.3f}".format(eval_precision/nb_eval_steps))
print("Recall: {0:.3f}".format(eval_recall/nb_eval_steps))
print("Accuracy: {0:.3f}".format(eval_accuracy/nb_eval_steps))
print("f1_score: {0:.3f}".format(eval_f1score/nb_eval_steps))
print("Test took: {:}".format(format_time(time.time() - t0)))


Precision: 0.836
Recall: 0.888
Accuracy: 0.844
f1_score: 0.857
Test took: 0:00:02


In [ ]:
"""
-논문에서 공개된 성능 평가-
Precision: 0.763
Recall: 0.873
Accuracy: 0.768
F1-Score: 0.814
"""

In [ ]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [ ]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [ ]:
logits = test_sentences(['The people are not satisfied, but they will say that they have worked hard to find a way to solve the job problem.'])

print(logits)
print(np.argmax(logits))

[[-1.5992962  1.46809  ]]
1


In [ ]:
logits = test_sentences(['Hello my name is Taekhyun'])

print(logits)
print(np.argmax(logits))

[[ 0.39324045 -0.5058099 ]]
0


In [ ]:
logits = test_sentences(['The distrust that Korea\'s policy may change when the administration of Korea changes has been settled in Japan, so Japan Merely Looks On regarding the progress of the Korea-Japan relations.'])

print(logits)
print(np.argmax(logits))


[[-2.3004584  2.6102304]]
1
